## Segment Tree?
Problem Description

Given an array A of size N and Q queries. Perform following queries:

1 V 0 append V in the back of array.

2 X V set A[X] = V.

3 X 0 delete A[X]. Note: All element at back of X move forward to occupy void.

4 X Y find sum in range [X, Y].
### NOTE: 
For the query of type 4 X Y, output the sum % 109 + 7.
### Problem Constraints
1 <= N,Q <= 100000

1 <= A[i],V <=100000

1 <= X,Y <= N' Where, N' is current size of array.
### Input Format
First argument contains an integer array A.

Second argument contains a Q x 3 Matrix B.
### Output Format
Return an integer array containing answer to all query of type 4 X Y in chronological order.
### Example Input
 A = [1, 2, 5, 3, 4] 
 
 B = [ 
       
       [4, 2, 4], 
       
       [3, 3, 0],
       
       [1, 6, 0],
       
       [4, 3, 5] 
       
   ]
### Example Output
 [10, 13]
### Example Explanation
 First Query find sum(A[2],A[3],A[4])
 
 Second Query make A = [1, 2, 3, 4]
 
 Third Query make A = [1, 2, 3, 4, 6]
 
 Fourth Query find sum(A[3],A[4],A[5])

In [10]:
def buildTree(a):
    n = len(a)
    tree = [-1 for i in range(4*n)]
    #print(len(tree))
    def build(s, e, node_pos):
        #print(s,e,node_pos)
        if s == e:
            #print('ooo', s, e, node_pos)
            tree[node_pos] = a[s]
            #print(tree)
            return
        m = (s+e)//2
        #print('yess', s, m, e, node_pos)
        build(s, m, 2*node_pos+1)
        build(m+1, e, 2*node_pos+2)
        #print('nooo', tree)
        tree[node_pos] = tree[2*node_pos+1] + tree[2*node_pos+2]
    build(0, n-1, 0)
    return tree

def rangeQuery(tree, a, qL, qR):
    n = len(a)
    def query(qL, qR, s, e, node_pos):
        if qL <= s and qR >= e:
            return tree[node_pos]
        if qL > e or qR < s:
            return 0
        m = (s+e)//2
        return query(qL, qR, s, m, 2*node_pos+1) + query(qL, qR, m+1, e, 2*node_pos+2)
    return query(qL, qR, 0, n-1, 0)

def updateQuery(tree, a, idx, val):
    n = len(a)
    def update(idx, val, s, e, node_pos):
        if s == e:
            a[idx] = val
            tree[node_pos] = val
        else:
            m = (s+e)//2
            if s <= idx and idx <= m:
                update(idx, val, s, m, 2*node_pos+1)
            else:
                update(idx, val, m+1, e, 2*node_pos+2)
            tree[node_pos] = tree[2*node_pos+1] + tree[2*node_pos+2]
    update(idx, val, 0, n-1, 0)
    
class Solution:
    # @param A : list of integers
    # @param B : list of list of integers
    # @return a list of integers
    def solve(self, a, b):
        res = []
        m = 10**9 + 7                                          
        tree = buildTree(a)
        for i in range(len(b)):
            if b[i][0] == 4:
                res.append(rangeQuery(tree, a, b[i][1]-1, b[i][2]-1))
            elif b[i][0] == 1:
                a.append(b[i][1])
                tree = buildTree(a)                  
                #print(tree,a)
            elif b[i][0] == 3:
                a.pop(b[i][1]-1)
                if len(a):
                    tree = buildTree(a)
                #print(tree,a)
            else:
                updateQuery(tree, a, b[i][1]-1, b[i][2])
        #print(rangeQuery(tree, a, 0, 4))
        return res    
        
o = Solution()
A = [ 2 ]
B = [
  [3, 1, 0],
  [1, 4, 0],
  [4, 1, 1]
]
print(o.solve(A,B))

[4]


In [13]:
class Solution:
    def createTree(self, st, A, s, e, sind):
        if s == e:
            st[sind] = A[s]
            return
        mid = (s+e)//2
        self.createTree(st, A, s, mid, 2*sind+1)
        self.createTree(st, A, mid+1, e, 2*sind+2)
        st[sind] = st[sind*2+1]+st[sind*2+2]
        return
    
    def getRange(self, st,  s, e, qs, qe, sind):
        if qs <= s and qe >= e:
            return st[sind]
        if qe < s or qs > e:
            return 0
        mid = (s+e)//2
        left = self.getRange(st, s, mid, qs, qe, 2*sind+1)
        right = self.getRange(st, mid+1, e, qs, qe, 2*sind+2)
        return (left+right)       

    def update(self, st, s, e, sind, ind, val):
        if s == e:
            st[sind] = val
            return
        mid = (s+e)//2
        if ind <= mid:
            self.update(st, s, mid, 2*sind+1, ind, val)
        else:
            self.update(st,  mid+1, e, 2*sind+2, ind, val)
        st[sind] = st[sind*2+1]+st[sind*2+2]

    def getindex(self, st, s, e, sind, i):
        if s == e:
            return s
        leftOnes = st[2*sind+1]
        mid = (s+e)//2
        if i <= leftOnes:
            return self.getindex(st, s, mid, 2*sind+1, i)
        else:
            return self.getindex(st, mid+1, e, 2*sind+2, i-leftOnes)
       
    def solve(self, A, B):
        m = 10**9+7
        n1 = len(A)
        n = len(A)+len(B)
        A = A+[0]*len(B)
        st = [0]*(4*n)
        de = [1]*n
        dest = [0]*(4*n)
        self.createTree(st, A, 0, n-1, 0)
        self.createTree(dest, de, 0, n-1, 0)
        ans = []
        for q in B:
            if q[0] == 1:
                temp = self.getindex(dest, 0, n-1, 0, n1+1)
                self.update(st,0, n-1, 0, temp, q[1])
                n1 += 1
                A[temp] = q[1]
            if q[0] == 2:
                if 1 <= q[1]<=n1:
                    temp = self.getindex(dest, 0, n-1,0, q[1])
                    self.update(st,0, n-1, 0, temp, q[2])
                    A[temp] = q[2]
            if q[0] == 3:
                if 1 <= q[1]<=n1:
                    temp = self.getindex(dest, 0, n-1,0, q[1])
                    self.update(st,0, n-1, 0, temp, 0)
                    self.update(dest, 0, n-1, 0, temp, 0)
                    de[temp] = 0
                    n1 -= 1
                    A[temp] = 0                   
            if q[0] == 4:
                qs = self.getindex(dest, 0, n-1,0, q[1])
                qe = self.getindex(dest, 0, n-1,0, q[2])
                ans.append(self.getRange(st,  0, n-1, qs, qe, 0)%m)
        # print(A)
        return ans
o = Solution()
A = [1, 2, 5, 3, 4]

B = [

   [4, 2, 4], 

   [3, 3, 0],

   [1, 6, 0],

   [4, 3, 5] 
]
print(o.solve(A,B))

[10, 13]


In [ ]:
BIT = [[0 for i in range(200005)] for j in range(2)]
ar = [0] * 200005
n = 0 
t = 0
maxN = 200005
mod = 1e9 +7
def update( i, x, delta):
    global BIT, maxN  
    while(x < maxN):
        BIT[i][x] += delta
        x += x&(-x)
    
def query( i, x):
    global BIT, maxN  
    summ = 0
    while(x > 0):
        summ += BIT[i][x]
        x -= x & (-x)
    return summ    
def getidx( x):
    lo=1
    hi=n
    idx=0
    while(lo<=hi):
        mid = (lo+hi)//2
        qu=query(1,mid)
        if((x+qu)==mid):
            idx=mid
            hi=mid-1
        elif((x+qu)<mid):
            hi=mid-1
        else:
            lo=mid+1
    return idx
    
class Solution:
    # @param A : list of integers
    # @param B : list of list of integers
    # @return a list of integers
    def solve(self, A, B):
        global BIT, ar,maxN,n,t
        n= len(A)
        BIT = [[0 for i in range(200005)] for j in range(2)]
        ar = [0] * 200005
        for i in range(1,n+1):
            ar[i]=A[i-1]
            update(0,i,ar[i])
        ans = []
        for i in B:
            t=i[0]
            if(t==1):
                n +=1
                ar[n]=i[1]
                update(0,n,ar[n])
            if(t==2):    # update ar[x] to Y
                x=i[1]
                y=i[2]
                idx=getidx(x)
                update(0,idx,y-ar[idx])
                ar[idx]=y
            
            if(t==3):    # delete ar[x]
                x=i[1]
                idx=getidx(x);
                update(0,idx,-ar[idx])
                update(1,idx,1)
            if(t==4) :   # query sum in [X,Y]
                idx1=0
                idx2=0
                x=i[1]
                y=i[2]
                idx1 = getidx(x);
                idx2 = getidx(y);
                val = query(0,idx2)-query(0,idx1-1)
                val = val %mod;
                val = int(val)
                ans.append(val)
        return ans